In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [6]:
df = pd.read_excel("/content/drive/MyDrive/AutomatedQuestionGeneration/train.xlsx")

In [7]:
df = df.drop("Unnamed: 0",axis = 1)

In [8]:
df.head()

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0


In [ ]:
!pip3 install bertopic[all]

In [1]:
from bertopic import BERTopic

In [9]:
unique_contexts = df["context"].unique()

In [10]:
topic_model = BERTopic(language="english", calculate_probabilities=True) # We need the probabilities to visualize
topics, _ = topic_model.fit_transform(unique_contexts)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [11]:
# Get the most frequent topics
topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
print(f"{outliers} documents have not been classified")
print(f"The other {topic_freq['Count'].sum() - outliers} documents are {topic_freq['Topic'].shape[0]-1} topics")

3922 documents have not been classified
The other 15107 documents are 389 topics


In [12]:
for i in range(10):
  topic_no = topic_freq['Topic'].iloc[i]
  print("Topic ", topic_no," : \n")
  print(topic_model.get_topic(topic_no))
  print("\n\n")

Topic  -1  : 

[('greek', 0.0012663037164139324), ('government', 0.0012037935239505705), ('war', 0.0011743848520761985), ('american', 0.0011603064926920337), ('european', 0.00110046490211792), ('german', 0.001097077141666627), ('national', 0.0010861542546285361), ('british', 0.0010823011209823035), ('political', 0.0010724412218595005), ('americans', 0.001039994296718821)]



Topic  0  : 

[('presbyterian', 0.020318409270686653), ('protestant', 0.016676082459583307), ('catholic', 0.011294566983368034), ('protestants', 0.007168762179763483), ('evangelical', 0.007101946166151343), ('presbyterians', 0.006033104901183904), ('christian', 0.00601288849837271), ('christianity', 0.005686398467722067), ('christians', 0.005220550277664412), ('protestantism', 0.004965458599781087)]



Topic  1  : 

[('architecture', 0.01723857044889006), ('cathedral', 0.00992889447284525), ('buildings', 0.009543265455646694), ('building', 0.007991448276641944), ('romanesque', 0.006711317643084604), ('architects', 

In [13]:
# Returns how spatially separated each topic is.
topic_model.visualize_topics()

In [14]:
import re
import numpy as np

In [15]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [49]:
def Sort_Tuple(tup):
  # Sorting a list of tuples using the second item  
    tup.sort(key = lambda x: x[1]) 
    return tup 

split_ratio = 0.6

def reference_bifurcate_tm(original_context,answer_start,index):  # ... 
  # Total sentences in the original_context
  #n_sentences = original_context.count('. ') + 1
  original_context += " "
  #p = re.compile('\w\. ')
  p = re.compile("[a-z0-9|\"|\'\)]*\. ")
  #p = re.compile("( \w\. ){6,}")
  n_sentences = len(p.findall(original_context)) 
  if n_sentences <= 1:
    return np.nan,np.nan

  # Number of sentenecs to consider in the reference context --> Half the size of the given context
  window_size = int(np.round(n_sentences*split_ratio))
  #Perform NER on each sentence and select window_size number of sentences (which have the least number of Named Entities)
  #context_list = original_context.split(". ")
  #d = ". "
  #context_list =  [e+d for e in original_context.split(d) if e]
  context_list = re.split(p,original_context)
  l = p.findall(original_context)
  l.append("")
  context_list = [a+b for a,b in zip(re.split(p,original_context),l)]
  context_len_list = [len(x) for x in context_list]
  c = 0
  answer_sentence_index = 0
  for elem in context_len_list:
    c += elem
    if c >= answer_start:
      break
    answer_sentence_index += 1
  
  print(n_sentences)
  print(answer_sentence_index)


  # # Initialising NER list counter
  # ner_count = []


  # for i in range(n_sentences):
  #   # Appending the number of entities of each sentence to a list
  #   ner_count.append((i,len(nlp(context_list[i]).ents)))
  
  # ner_count = Sort_Tuple(ner_count)

  ref_context = []
  given_context = []
  context_given = ""
  context_ref = ""

  
  j = 0
  topic_similarity_average = []
  #ref_context = ""
  #given_context = ""

  if (index % 50) == 0:
    #given_context += context_list[answer_sentence_index] 
    given_context.append((context_list[answer_sentence_index],answer_sentence_index))
    # print("G")
    # print(given_context)
    # window_size += 1
  else:
    #ref_context += context_list[answer_sentence_index]  # Answer sentence
    ref_context.append((context_list[answer_sentence_index],answer_sentence_index))
    # print("F")
    # print(ref_context)
    window_size -= 1
 

  answer_topic = topic_model.find_topics(context_list[answer_sentence_index])

  for j in range(n_sentences):
    if (context_list[j] == context_list[answer_sentence_index]):
      continue
    # if (j < window_size):
    #   ref_context = ". ".join([ref_context,context_list[ner_count[j][0]]])
    # else:
    #   given_context = ". ".join([given_context,context_list[ner_count[j][0]]])
    else:
      temp = topic_model.find_topics(context_list[j])
      # Finding the matching topics to compute the corresponding similarity average
      temp_similarty_list = [key for key, val in enumerate(temp[0]) if val in set(answer_topic[0])]
      s = 0
      for item in temp_similarty_list:
        s += temp[1][item]
      if len(temp_similarty_list) == 0:
        s = 0
      else:  
        s /= len(temp_similarty_list)
      topic_similarity_average.append((j,s))

  topic_similarity_average = Sort_Tuple(topic_similarity_average)

  for k in range(1,window_size + 1):
    # if k == 1:
    #   ref_context = context_list[topic_similarity_average[-k][0]]
    # else:
    #ref_context = "".join([ref_context,context_list[topic_similarity_average[-k][0]]])
    #print((context_list[topic_similarity_average[-k][0]],topic_similarity_average[-k][0]))
    ref_context.append((context_list[topic_similarity_average[-k][0]],topic_similarity_average[-k][0]))
  for y in range(window_size + 1, n_sentences):
    # if y == window_size + 1:
    #   given_context =  context_list[topic_similarity_average[-y][0]]
    # else:
    #given_context = "".join([given_context,context_list[topic_similarity_average[-y][0]]])
    #print((context_list[topic_similarity_average[-y][0]],topic_similarity_average[-y][0]))
    given_context.append((context_list[topic_similarity_average[-y][0]],topic_similarity_average[-y][0]))

  ref_context_inorder = Sort_Tuple(ref_context)
  given_context_inorder = Sort_Tuple(given_context)

  # print(given_context_inorder)
  # print()
  # print(ref_context_inorder)
  # print()
  


  for sentence,index in given_context_inorder:
    context_given = "".join([context_given,sentence])
  for sentence,index in ref_context_inorder:
    context_ref = "".join([context_ref,sentence])

  return context_given, context_ref

In [ ]:
#import spacy library
import spacy
  
#load core english library
nlp = spacy.load("en_core_web_sm")
  
#take unicode string  
#here u stands for unicode
doc = nlp(r"""The remaining band members recorded "Independent Women Part I", which appeared on the soundtrack to the 2000 film, Charlie\'s Angels. It became their best-charting single, topping the U.S. Billboard Hot 100 chart for eleven consecutive weeks. In early 2001, while Destiny\'s Child was completing their third album, Beyoncé landed a major role in the MTV made-for-television film, Carmen: A Hip Hopera, starring alongside American actor Mekhi Phifer. Set in Philadelphia, the film is a modern interpretation of the 19th century opera Carmen by French composer Georges Bizet. When the third album Survivor was released in May 2001, Luckett and Roberson filed a lawsuit claiming that the songs were aimed at them. The album debuted at number one on the U.S. Billboard 200, with first-week sales of 663,000 copies sold. The album spawned other number-one hits, "Bootylicious" and the title track, "Survivor", the latter of which earned the group a Grammy Award for Best R&B Performance by a Duo or Group with Vocals. After releasing their holiday album 8 Days of Christmas in October 2001, the group announced a hiatus to further pursue solo careers.""")
#to print sentences
for sent in doc.sents:
  print(sent)

In [51]:
temp.iloc[57944,2]

"St. John's (/ˌseɪntˈdʒɒnz/, local /ˌseɪntˈdʒɑːnz/) is the capital and largest city in Newfoundland and Labrador, Canada. St. John's was incorporated as a city in 1888, yet is considered by some to be the oldest English-founded city in North America. It is located on the eastern tip of the Avalon Peninsula on the island of Newfoundland. With a population of 214,285 as of July 1, 2015, the St. John's Metropolitan Area is the second largest Census Metropolitan Area (CMA) in Atlantic Canada after Halifax and the 20th largest metropolitan area in Canada. It is one of the world's top ten oceanside destinations, according to National Geographic Magazine. Its name has been attributed to the feast day of John the Baptist, when John Cabot was believed to have sailed into the harbour in 1497, and also to a Basque fishing town with the same name."

In [ ]:
# Function to Split Sentences
#https://github.com/apoudel1021/Sentence-Segmentation/blob/master/sentencesegmentation.py

In [96]:
# -*- coding: utf-8 -*-
"""
Created on Sun Sep 02 22:33:09 2018
@author: deadp
"""
#import csv 

'''  Took help from the internet/book/friend. Not solely my idea !!  '''

import re

'''  Initializing the prefixes and suffixes and else   '''
caps = "([A-Z])"
digits = "([0-9])"
lower = "([a-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|MR|MRS)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
caps = "([A-Z])"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](edu|com|net|org|io|gov)"
digits = "([0-9])"


def split_sentences(text):
#    print text
    text = " " + text + "  "
#    print text
    ''' Get the whole corpus removing all the new lines'''
    text = text.replace("\n"," ")
#    print text
    ''' Replace the dot on prefeixes with <use>  '''
    text = re.sub(prefixes,"\\1<use>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
#    print text
    ''' For the websites : Not really applicable for this project '''
    text = re.sub(websites,"<use>\\1",text)
    #print(text)
    ''' For Ph.D'''
#    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<use> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    
    '''A.B.C '''
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<use>\\2<use>\\3<use>",text)
    '''  A.B  '''
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<use>\\2<use>",text)
    ''' a.b, e.g, i.e'''
    text = re.sub(lower + "[.]" + lower + "[.]","\\1<use>\\2<use>",text)
    ''' i.e., e.g. '''
    text = re.sub(lower + "[.]" + lower + "[.]"+ lower + "[.]","\\1<use>\\2<use>",text)
    
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<use>",text)
    
    ''' .A.'''
    text = re.sub(" " + caps + "[.]"," \\1<use>",text)
    
    '''checking for special cases like ...., ? , ! '''
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<use>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    # Remove first character from sentence if its a space
    sentences = [sent[1:] for sent in sentences if sent[0] == " "]
    # Add a space at the end of each sentence
    sentences = [sent + " " for sent in sentences]
    return sentences

In [97]:
# Bifurcation function that uses the split_sentence function

In [98]:
def Sort_Tuple(tup):
  # Sorting a list of tuples using the second item  
    tup.sort(key = lambda x: x[1]) 
    return tup 

split_ratio = 0.6

def reference_bifurcate_custom(original_context,answer_start,index):  # ... 
  # Total sentences in the original_context
  #n_sentences = original_context.count('. ') + 1
  original_context += " "
  #p = re.compile('\w\. ')
  #p = re.compile("[a-z0-9|\"|\'\)]*\. ")
  #p = re.compile("( \w\. ){6,}")
  

  
  #Perform NER on each sentence and select window_size number of sentences (which have the least number of Named Entities)
  #context_list = original_context.split(". ")
  #d = ". "
  #context_list =  [e+d for e in original_context.split(d) if e]
  #context_list = re.split(p,original_context)
  #l = p.findall(original_context)
  #l.append("")
  context_list = split_sentences(original_context)
  context_len_list = [len(x) for x in context_list]
  n_sentences = len(context_list) 
  if n_sentences <= 1:
    return np.nan,np.nan
  # Number of sentenecs to consider in the reference context --> Half the size of the given context
  window_size = int(np.round(n_sentences*split_ratio))
  c = 0
  answer_sentence_index = 0
  for elem in context_len_list:
    c += elem
    if c >= answer_start:
      break
    answer_sentence_index += 1
  
  print(n_sentences)
  print(answer_sentence_index)


  # # Initialising NER list counter
  # ner_count = []


  # for i in range(n_sentences):
  #   # Appending the number of entities of each sentence to a list
  #   ner_count.append((i,len(nlp(context_list[i]).ents)))
  
  # ner_count = Sort_Tuple(ner_count)

  ref_context = []
  given_context = []
  context_given = ""
  context_ref = ""

  
  j = 0
  topic_similarity_average = []
  #ref_context = ""
  #given_context = ""

  if (index % 50) == 0:
    #given_context += context_list[answer_sentence_index] 
    given_context.append((context_list[answer_sentence_index],answer_sentence_index))
    # print("G")
    # print(given_context)
    # window_size += 1
  else:
    #ref_context += context_list[answer_sentence_index]  # Answer sentence
    ref_context.append((context_list[answer_sentence_index],answer_sentence_index))
    # print("F")
    # print(ref_context)
    window_size -= 1
 

  answer_topic = topic_model.find_topics(context_list[answer_sentence_index])

  for j in range(n_sentences):
    if (context_list[j] == context_list[answer_sentence_index]):
      continue
    # if (j < window_size):
    #   ref_context = ". ".join([ref_context,context_list[ner_count[j][0]]])
    # else:
    #   given_context = ". ".join([given_context,context_list[ner_count[j][0]]])
    else:
      temp = topic_model.find_topics(context_list[j])
      # Finding the matching topics to compute the corresponding similarity average
      temp_similarty_list = [key for key, val in enumerate(temp[0]) if val in set(answer_topic[0])]
      s = 0
      for item in temp_similarty_list:
        s += temp[1][item]
      if len(temp_similarty_list) == 0:
        s = 0
      else:  
        s /= len(temp_similarty_list)
      topic_similarity_average.append((j,s))

  topic_similarity_average = Sort_Tuple(topic_similarity_average)

  for k in range(1,window_size + 1):
    # if k == 1:
    #   ref_context = context_list[topic_similarity_average[-k][0]]
    # else:
    #ref_context = "".join([ref_context,context_list[topic_similarity_average[-k][0]]])
    #print((context_list[topic_similarity_average[-k][0]],topic_similarity_average[-k][0]))
    ref_context.append((context_list[topic_similarity_average[-k][0]],topic_similarity_average[-k][0]))
  for y in range(window_size + 1, n_sentences):
    # if y == window_size + 1:
    #   given_context =  context_list[topic_similarity_average[-y][0]]
    # else:
    #given_context = "".join([given_context,context_list[topic_similarity_average[-y][0]]])
    #print((context_list[topic_similarity_average[-y][0]],topic_similarity_average[-y][0]))
    given_context.append((context_list[topic_similarity_average[-y][0]],topic_similarity_average[-y][0]))

  ref_context_inorder = Sort_Tuple(ref_context)
  given_context_inorder = Sort_Tuple(given_context)

  # print(given_context_inorder)
  # print()
  # print(ref_context_inorder)
  # print()
  


  for sentence,index in given_context_inorder:
    context_given = "".join([context_given,sentence])
  for sentence,index in ref_context_inorder:
    context_ref = "".join([context_ref,sentence])

  return context_given, context_ref

In [115]:
g,r = reference_bifurcate_custom(temp.iloc[0,2],temp.iloc[0,4],0)

4
1


In [116]:
g

"Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. "

In [117]:
r

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". '

In [114]:
temp.iloc[57944,2]

"St. John's (/ˌseɪntˈdʒɒnz/, local /ˌseɪntˈdʒɑːnz/) is the capital and largest city in Newfoundland and Labrador, Canada. St. John's was incorporated as a city in 1888, yet is considered by some to be the oldest English-founded city in North America. It is located on the eastern tip of the Avalon Peninsula on the island of Newfoundland. With a population of 214,285 as of July 1, 2015, the St. John's Metropolitan Area is the second largest Census Metropolitan Area (CMA) in Atlantic Canada after Halifax and the 20th largest metropolitan area in Canada. It is one of the world's top ten oceanside destinations, according to National Geographic Magazine. Its name has been attributed to the feast day of John the Baptist, when John Cabot was believed to have sailed into the harbour in 1497, and also to a Basque fishing town with the same name."

In [102]:
g1,r1 = reference_bifurcate_tm(temp.iloc[125,2],temp.iloc[125,4],125)

7
0


In [108]:
g1

'Billboard Hot 100 chart. Her first solo album Dangerously in Love was released on June 24, 2003, after Michelle Williams and Kelly Rowland had released their solo efforts. The single "Baby Boy" also reached number one, and singles, "Me, Myself and I" and "Naughty Girl", both reached the top-five. '

In [109]:
r1

'Beyoncé\'s first solo recording was a feature on Jay Z\'s "\'03 Bonnie & Clyde" that was released in October 2002, peaking at number four on the U.S. The album sold 317,000 copies in its first week, debuted atop the Billboard 200, and has since sold 11 million copies worldwide. The album\'s lead single, "Crazy in Love", featuring Jay Z, became Beyoncé\'s first number-one single as a solo artist in the US. The album earned Beyoncé a then record-tying five awards at the 46th Annual Grammy Awards; Best Contemporary R&B Album, Best Female R&B Vocal Performance for "Dangerously in Love 2", Best R&B Song and Best Rap/Sung Collaboration for "Crazy in Love", and Best R&B Performance by a Duo or Group with Vocals for "The Closer I Get to You" with Luther Vandross. '

In [63]:
temp.iloc[101,2]

'The remaining band members recorded "Independent Women Part I", which appeared on the soundtrack to the 2000 film, Charlie\'s Angels. It became their best-charting single, topping the U.S. Billboard Hot 100 chart for eleven consecutive weeks. In early 2001, while Destiny\'s Child was completing their third album, Beyoncé landed a major role in the MTV made-for-television film, Carmen: A Hip Hopera, starring alongside American actor Mekhi Phifer. Set in Philadelphia, the film is a modern interpretation of the 19th century opera Carmen by French composer Georges Bizet. When the third album Survivor was released in May 2001, Luckett and Roberson filed a lawsuit claiming that the songs were aimed at them. The album debuted at number one on the U.S. Billboard 200, with first-week sales of 663,000 copies sold. The album spawned other number-one hits, "Bootylicious" and the title track, "Survivor", the latter of which earned the group a Grammy Award for Best R&B Performance by a Duo or Group

In [ ]:
g, r = reference_bifurcate_tm(temp["context"][57944],temp["answer_start"][57944],57944)

In [45]:
g

nan

In [46]:
r

nan

In [39]:
temp.iloc[101,2]

'The remaining band members recorded "Independent Women Part I", which appeared on the soundtrack to the 2000 film, Charlie\'s Angels. It became their best-charting single, topping the U.S. Billboard Hot 100 chart for eleven consecutive weeks. In early 2001, while Destiny\'s Child was completing their third album, Beyoncé landed a major role in the MTV made-for-television film, Carmen: A Hip Hopera, starring alongside American actor Mekhi Phifer. Set in Philadelphia, the film is a modern interpretation of the 19th century opera Carmen by French composer Georges Bizet. When the third album Survivor was released in May 2001, Luckett and Roberson filed a lawsuit claiming that the songs were aimed at them. The album debuted at number one on the U.S. Billboard 200, with first-week sales of 663,000 copies sold. The album spawned other number-one hits, "Bootylicious" and the title track, "Survivor", the latter of which earned the group a Grammy Award for Best R&B Performance by a Duo or Group

In [183]:
temp.iloc[101]

index                                    56be8e353aeaaa14008c90c8
question        For what network, did Beyonce land a major mov...
context         The remaining band members recorded "Independe...
text                                                          MTV
answer_start                                                  348
c_id                                                            8
Name: 101, dtype: object

In [57]:
print(("a",1))

('a', 1)


In [58]:
print([(1,1.5),(2,2.5),(1,1.1)])

[(1, 1.5), (2, 2.5), (1, 1.1)]


In [20]:
Sort_Tuple([(1,1.5),(2,2.5),(1,1.1)])

[(1, 1.1), (1, 1.5), (2, 2.5)]

In [ ]:
topic_similarity_average = [(3,0.45),(5,0.53),(1,0.65),(4,0.77)] --->2

In [ ]:
context_list = ["abc. ","def. ","ghi. ","jkl. ","mno. "]

In [ ]:
ref = "def. jkl. abc. " --> "abc. def. jkl. "
given = "mno. ghi. " --> "ghi. mno. "

In [ ]:
ref -->3
given -->2

In [ ]:
p = re.compile('\w\. ')
context_list = re.split(p,df.iloc[3,2])
l = p.findall(df.iloc[3,2])
l.append("")
context_list_temp = [a+b for a,b in zip(re.split(p,df.iloc[3,2]),l)]
context_len_list = [len(x) for x in context_list]

In [ ]:
re.split(p,"I am a disco dancer. ")

['I am a disco dance', '']

In [ ]:
df.iloc[3,2]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [ ]:
temp[temp["answer_start"] == "None"]
temp["answer_start"].unique()

array([ 269.,  207.,  526.,  166.,  276.,  320.,  505.,  360.,  290.,
        590.,  369.,  565.,  260.,  586.,  180.,  406.,   48.,   95.,
        466.,  104.,  935.,  985.,  736.,   18.,  970.,  393.,  445.,
        552.,  303.,  204.,  330.,  578.,  152.,  101.,  255.,  540.,
        117.,   49.,  165.,  507.,  148.,  711.,  484.,  385.,  355.,
        542.,  918.,  537., 1264.,    3.,    7.,  192., 1126.,  215.,
        848., 1212.,   51.,   85.,  849.,  688.,  169.,  714.,  194.,
        396.,  110.,  149.,   37.,  216.,  348.,  793.,  557.,  593.,
        115.,  378.,  628., 1070.,   84.,  331.,  431.,  705.,  834.,
        210.,  416.,  435.,  545.,   58.,  240.,  123.,  193.,  419.,
        474.,  130., 1042.,  229.,  696.,  513.,  664., 1206.,   38.,
        100.,  253.,  132.,  346.,  323.,  342.,   28.,  635.,   53.,
        171.,  550.,  671.,  112.,  576.,  932.,  436.,  487.,   34.,
        897., 1567., 1881.,   78., 1744.,  156.,  605., 1611.,   69.,
        439.,  582.,

In [18]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86820 entries, 0 to 86819
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         86820 non-null  object 
 1   question      86820 non-null  object 
 2   context       86820 non-null  object 
 3   text          86820 non-null  object 
 4   answer_start  86820 non-null  float64
 5   c_id          86820 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 4.0+ MB


In [118]:
temp = df.dropna(axis = 0)
temp = temp.reset_index(drop=True)
temp.head(100)

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0
...,...,...,...,...,...,...
95,56d462f82ccc5a1400d8311f,Who replaced Luckett and Roberson in Destiny's...,LeToya Luckett and Roberson became unhappy wit...,Farrah Franklin and Michelle Williams.,110.0,7
96,56d462f82ccc5a1400d83120,Who was blamed for Luckett and Roberson leavin...,LeToya Luckett and Roberson became unhappy wit...,Beyoncé,149.0,7
97,56d462f82ccc5a1400d83121,Who helped Beyoncé overcome her depression dur...,LeToya Luckett and Roberson became unhappy wit...,her mother,714.0,7
98,56d462f82ccc5a1400d83123,Which newest member was removed from Destiny's...,LeToya Luckett and Roberson became unhappy wit...,Farrah Franklin,110.0,7


In [ ]:
original_context = df.iloc[0,2] + " "
original_context

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". '

In [ ]:
original_context[-1]

' '

In [ ]:
c1 = df.iloc[0,2]
n_sentences = len(p.findall(c1)) 
print(n_sentences)

3


In [ ]:
c1 = df.iloc[0,2] + " "
n_sentences = len(p.findall(c1)) 
print(n_sentences)

3


In [ ]:
"".join(context_list) == original_context

True

In [ ]:
p = re.compile('\w\. ')
n_sentences = len(p.findall(original_context)) 
print(n_sentences)
# Number of sentenecs to consider in the reference context --> Half the size of the given context
#window_size = int(np.round(n_sentences*split_ratio))
#Perform NER on each sentence and select window_size number of sentences (which have the least number of Named Entities)
#context_list = original_context.split(". ")
#d = ". "
#context_list =  [e+d for e in original_context.split(d) if e]
context_list = re.split(p,original_context)
l = p.findall(original_context)
l.append("")
context_list = [a+b for a,b in zip(re.split(p,original_context),l)]
context_len_list = [len(x) for x in context_list]

3


In [ ]:
p = re.compile("[\w|\"|\']\. ")
text = "abc. def. gef\". "
p.findall(text)

['c. ', 'f. ', '". ']

In [ ]:
print(text)

abc. def. gef". 


In [ ]:
context_list

['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. ',
 "Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. ",
 "Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. ",
 'Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". ']

In [19]:
temp["given_context"] = "a"
temp["reference_context"] = "a"

In [ ]:
# Index to start from 57944 til end

In [20]:
from tqdm.notebook import tqdm
for i in tqdm(range(len(temp))):
  temp["given_context"][i],temp["reference_context"][i] = reference_bifurcate_custom(temp["context"][i],temp["answer_start"][i],i)
#temp["given_context"] = temp["given_context"]
#temp["reference_context"] = temp["reference_context"]

  0%|          | 0/86820 [00:00<?, ?it/s]

IndexError: ignored

In [31]:
temp.iloc[57944,2]

"St. John's (/ˌseɪntˈdʒɒnz/, local /ˌseɪntˈdʒɑːnz/) is the capital and largest city in Newfoundland and Labrador, Canada. St. John's was incorporated as a city in 1888, yet is considered by some to be the oldest English-founded city in North America. It is located on the eastern tip of the Avalon Peninsula on the island of Newfoundland. With a population of 214,285 as of July 1, 2015, the St. John's Metropolitan Area is the second largest Census Metropolitan Area (CMA) in Atlantic Canada after Halifax and the 20th largest metropolitan area in Canada. It is one of the world's top ten oceanside destinations, according to National Geographic Magazine. Its name has been attributed to the feast day of John the Baptist, when John Cabot was believed to have sailed into the harbour in 1497, and also to a Basque fishing town with the same name."

In [32]:
temp.iloc[57944,6]

'a'

In [33]:
% debug

> <ipython-input-16-2ddceede0318>(113)reference_bifurcate_tm()
    111     #given_context = "".join([given_context,context_list[topic_similarity_average[-y][0]]])
    112     #print((context_list[topic_similarity_average[-y][0]],topic_similarity_average[-y][0]))
--> 113     given_context.append((context_list[topic_similarity_average[-y][0]],topic_similarity_average[-y][0]))
    114 
    115   ref_context_inorder = Sort_Tuple(ref_context)

ipdb> i
57944
ipdb> n_sentences
9
--KeyboardInterrupt--
ipdb> exit


In [ ]:
temp

In [27]:
temp.head(57944)

,index,question,context,text,answer_start,c_id,given_context,reference_context
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
...,...,...,...,...,...,...,...,...
57939,572835132ca10214002da0be,When were the first supplemental LTRO's offered?,The ECB's first supplementary longer-term refi...,March 2008,111.0,12734,It announced two 3-month and one 6-month full ...,The ECB's first supplementary longer-term refi...
57940,572835132ca10214002da0bf,How long had the time to maturity been previou...,The ECB's first supplementary longer-term refi...,three months,165.0,12734,It announced two 3-month and one 6-month full ...,The ECB's first supplementary longer-term refi...
57941,572835132ca10214002da0c0,How many bids were recorded at the auction for...,The ECB's first supplementary longer-term refi...,close to 1100 bidders,568.0,12734,Previously the longest tender offered was thre...,The ECB's first supplementary longer-term refi...
57942,5727fa9c3acd2414000df147,In what country is St. John's located?,"St. John's (/ˌseɪntˈdʒɒnz/, local /ˌseɪntˈdʒɑː...",Canada,113.0,12735,"John's was incorporated as a city in 1888, yet...","St. John's (/ˌseɪntˈdʒɒnz/, local /ˌseɪntˈdʒɑː..."


In [28]:
temp_1 = temp[:57944]
temp_1.to_csv("/content/drive/MyDrive/AutomatedQuestionGeneration/Topic_Bifurcated_SQUAD1.csv",index=None)

In [192]:
temp.head(10)

,index,question,context,text,answer_start,c_id,given_context,reference_context
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
5,56bf6b0f3aeaaa14008c9603,In what R&B group was she the lead singer?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Destiny's Child,320.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
6,56bf6b0f3aeaaa14008c9604,What album made her a worldwide known artist?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Dangerously in Love,505.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
7,56bf6b0f3aeaaa14008c9605,Who managed the Destiny's Child group?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Mathew Knowles,360.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Managed by her father, Mathew Knowles, the gro..."
8,56d43c5f2ccc5a1400d830a9,When did Beyoncé rise to fame?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
9,56d43c5f2ccc5a1400d830aa,What role did Beyoncé have in Destiny's Child?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,lead singer,290.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."


In [126]:
g, r = reference_bifurcate_tm(temp["context"][25],temp["answer_start"][25],25)

In [127]:
g

'Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of darker themes. Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". '

In [128]:
r

'Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of darker themes. '

In [129]:
temp.iloc[25,2]

'Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploratio

In [130]:
temp.iloc[25]

index                                         56d43da72ccc5a1400d830c0
question                                    Who is Beyoncé married to?
context              Following the disbandment of Destiny's Child i...
text                                                             Jay Z
answer_start                                                       369
c_id                                                                 1
given_context        Beyoncé also ventured into acting, with a Gold...
reference_context    Following the disbandment of Destiny's Child i...
Name: 25, dtype: object

In [100]:
temp.head()

,index,question,context,text,answer_start,c_id,given_context,reference_context
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0,Their hiatus saw the release of Beyoncé's debu...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...


In [99]:
temp.iloc[]

index                                         56be8e353aeaaa14008c90c8
question             For what network, did Beyonce land a major mov...
context              The remaining band members recorded "Independe...
text                                                               MTV
answer_start                                                       348
c_id                                                                 8
given_context        It became their best-charting single, topping ...
reference_context    The remaining band members recorded "Independe...
Name: 101, dtype: object

In [101]:
temp.iloc[0,2]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [102]:
# Given
temp.iloc[0,6]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". '

In [103]:
# Reference
temp.iloc[0,7]

"Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. "

In [76]:
%debug

> <ipython-input-69-246fe75926c7>(103)reference_bifurcate_tm()
    101     #given_context = "".join([given_context,context_list[topic_similarity_average[-y][0]]])
    102     #print((context_list[topic_similarity_average[-y][0]],topic_similarity_average[-y][0]))
--> 103     given_context.append((context_list[topic_similarity_average[-y][0]],topic_similarity_average[-y][0]))
    104 
    105   ref_context_inorder = Sort_Tuple(ref_context)

ipdb> i
315
ipdb> i
315
--KeyboardInterrupt--
ipdb> exit



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



In [ ]:
df.iloc[550,2]

'The Bey Hive is the name given to Beyoncé\'s fan base. Fans were previously titled "The Beyontourage", (a portmanteau of Beyoncé and entourage). The name Bey Hive derives from the word beehive, purposely misspelled to resemble her first name, and was penned by fans after petitions on the online social networking service Twitter and online news reports during competitions.'

In [ ]:
 a = reference_bifurcate_tm(temp["context"][140],temp["answer_start"][140],140)

In [ ]:
df.loc[140]

index                                    56be91b23aeaaa14008c90f2
question        Destiny's Child got a star on the Hollywood Wa...
context         In November 2003, she embarked on the Dangerou...
text                                                         2006
answer_start                                                 1212
c_id                                                           11
Name: 140, dtype: object

In [ ]:
df.iloc[140,2]

'In November 2003, she embarked on the Dangerously in Love Tour in Europe and later toured alongside Missy Elliott and Alicia Keys for the Verizon Ladies First Tour in North America. On February 1, 2004, Beyoncé performed the American national anthem at Super Bowl XXXVIII, at the Reliant Stadium in Houston, Texas. After the release of Dangerously in Love, Beyoncé had planned to produce a follow-up album using several of the left-over tracks. However, this was put on hold so she could concentrate on recording Destiny Fulfilled, the final studio album by Destiny\'s Child. Released on November 15, 2004, in the US and peaking at number two on the Billboard 200, Destiny Fulfilled included the singles "Lose My Breath" and "Soldier", which reached the top five on the Billboard Hot 100 chart. Destiny\'s Child embarked on a worldwide concert tour, Destiny Fulfilled... and Lovin\' It and during the last stop of their European tour, in Barcelona on June 11, 2005, Rowland announced that Destiny\'s

In [ ]:
d = ". "
s =  [e+d for e in df.iloc[140,2].split(d) if e]

In [ ]:
s

['In November 2003, she embarked on the Dangerously in Love Tour in Europe and later toured alongside Missy Elliott and Alicia Keys for the Verizon Ladies First Tour in North America. ',
 'On February 1, 2004, Beyoncé performed the American national anthem at Super Bowl XXXVIII, at the Reliant Stadium in Houston, Texas. ',
 'After the release of Dangerously in Love, Beyoncé had planned to produce a follow-up album using several of the left-over tracks. ',
 "However, this was put on hold so she could concentrate on recording Destiny Fulfilled, the final studio album by Destiny's Child. ",
 'Released on November 15, 2004, in the US and peaking at number two on the Billboard 200, Destiny Fulfilled included the singles "Lose My Breath" and "Soldier", which reached the top five on the Billboard Hot 100 chart. ',
 "Destiny's Child embarked on a worldwide concert tour, Destiny Fulfilled... ",
 "and Lovin' It and during the last stop of their European tour, in Barcelona on June 11, 2005, Rowla

In [ ]:
d = ". "
context_list =  [e+d for e in df.iloc[140,2].split(d) if e]
context_len_list = [len(x) for x in context_list]

In [ ]:
context_len_list

[182, 133, 130, 130, 219, 75, 193, 157]

In [ ]:
print(temp.iloc[140,2])

In November 2003, she embarked on the Dangerously in Love Tour in Europe and later toured alongside Missy Elliott and Alicia Keys for the Verizon Ladies First Tour in North America. On February 1, 2004, Beyoncé performed the American national anthem at Super Bowl XXXVIII, at the Reliant Stadium in Houston, Texas. After the release of Dangerously in Love, Beyoncé had planned to produce a follow-up album using several of the left-over tracks. However, this was put on hold so she could concentrate on recording Destiny Fulfilled, the final studio album by Destiny's Child. Released on November 15, 2004, in the US and peaking at number two on the Billboard 200, Destiny Fulfilled included the singles "Lose My Breath" and "Soldier", which reached the top five on the Billboard Hot 100 chart. Destiny's Child embarked on a worldwide concert tour, Destiny Fulfilled... and Lovin' It and during the last stop of their European tour, in Barcelona on June 11, 2005, Rowland announced that Destiny's Chil

In [ ]:
for sentence in context_list:
  print(sentence)

In November 2003, she embarked on the Dangerously in Love Tour in Europe and later toured alongside Missy Elliott and Alicia Keys for the Verizon Ladies First Tour in North America
On February 1, 2004, Beyoncé performed the American national anthem at Super Bowl XXXVIII, at the Reliant Stadium in Houston, Texas
After the release of Dangerously in Love, Beyoncé had planned to produce a follow-up album using several of the left-over tracks
However, this was put on hold so she could concentrate on recording Destiny Fulfilled, the final studio album by Destiny's Child
Released on November 15, 2004, in the US and peaking at number two on the Billboard 200, Destiny Fulfilled included the singles "Lose My Breath" and "Soldier", which reached the top five on the Billboard Hot 100 chart
Destiny's Child embarked on a worldwide concert tour, Destiny Fulfilled..
and Lovin' It and during the last stop of their European tour, in Barcelona on June 11, 2005, Rowland announced that Destiny's Child woul

In [ ]:
len(temp.iloc[140,2])

1217

In [ ]:
answer_sentence_index = 0
c = 0
for i in context_len_list:
  c += i
  if c >= 1212:
    break
  answer_sentence_index += 1

In [ ]:
%debug

> <ipython-input-114-817c1012606a>(45)reference_bifurcate_tm()
     43     window_size += 1
     44   else:
---> 45     ref_context += context_list[answer_sentence_index] # Answer sentence
     46     window_size -= 1
     47 

ipdb> i
140
ipdb> exit


In [ ]:
temp["context"][155]

'Beyoncé\'s second solo album B\'Day was released on September 5, 2006, in the US, to coincide with her twenty-fifth birthday. It sold 541,000 copies in its first week and debuted atop the Billboard 200, becoming Beyoncé\'s second consecutive number-one album in the United States. The album\'s lead single "Déjà Vu", featuring Jay Z, reached the top five on the Billboard Hot 100 chart. The second international single "Irreplaceable" was a commercial success worldwide, reaching number one in Australia, Hungary, Ireland, New Zealand and the United States. B\'Day also produced three other singles; "Ring the Alarm", "Get Me Bodied", and "Green Light" (released in the United Kingdom only).'

In [ ]:
a[0]

'. B\'Day also produced three other singles; "Ring the Alarm", "Get Me Bodied", and "Green Light" (released in the United Kingdom only).. The second international single "Irreplaceable" was a commercial success worldwide, reaching number one in Australia, Hungary, Ireland, New Zealand and the United States'

In [ ]:
a[1]

'The album\'s lead single "Déjà Vu", featuring Jay Z, reached the top five on the Billboard Hot 100 chart. It sold 541,000 copies in its first week and debuted atop the Billboard 200, becoming Beyoncé\'s second consecutive number-one album in the United States. Beyoncé\'s second solo album B\'Day was released on September 5, 2006, in the US, to coincide with her twenty-fifth birthday'

In [ ]:
context_list = temp.iloc[155,2].split(". ")

In [ ]:
context_list = temp.iloc[155,2].split(". ")
context_len_list = [len(x) for x in context_list]

In [ ]:
context_len_list

[122, 152, 103, 169, 132]

In [ ]:
temp.head(500)

,index,question,context,text,answer_start,c_id,given_context,reference_context
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,"Born and raised in Houston, Texas, she perform...",Their hiatus saw the release of Beyoncé's debu...
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0,"Born and raised in Houston, Texas, she perform...",Their hiatus saw the release of Beyoncé's debu...
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
...,...,...,...,...,...,...,...,...
495,56d4e17f2ccc5a1400d832e4,What are Beyoncé's backup singers called?,"In 2006, Beyoncé introduced her all-female tou...",The Mamas,216.0,41,"In 2006, Beyoncé introduced her all-female tou...","Her background singers, The Mamas, consist of ..."
496,56d4e17f2ccc5a1400d832e5,When did The Mamas make their debut?,"In 2006, Beyoncé introduced her all-female tou...",the 2006 BET Awards,343.0,41,The band have supported Beyoncé in most subseq...,They made their debut appearance at the 2006 B...
497,56bec3303aeaaa14008c9391,What characteristics has Beyonce received accl...,Beyoncé has received praise for her stage pres...,stage presence and voice,36.0,42,Jim Farber of the Daily News and Stephanie Cla...,Beyoncé has received praise for her stage pres...
498,56bec3303aeaaa14008c9393,Which former president of Def Jam called Beyon...,Beyoncé has received praise for her stage pres...,L.A. Reid,445.0,42,Jim Farber of the Daily News and Stephanie Cla...,According to Barbara Ellen of The Guardian Bey...


In [ ]:
temp.iloc[0,2]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [ ]:
temp.iloc[0,1]

'When did Beyonce start becoming popular?'

In [ ]:
# Example 1: Given Context

In [ ]:
temp.iloc[0,6]

"Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. "

In [ ]:
# Exmaple 1: Reference Context

In [ ]:
temp.iloc[0,7]

'Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. '

In [ ]:
temp.to_csv("/content/drive/MyDrive/AutomatedQuestionGeneration/Topic-Model-Bifurcate.csv")

In [ ]:
temp = pd.read_csv("/content/drive/MyDrive/AutomatedQuestionGeneration/Topic-Model-Bifurcate.csv").drop("Unnamed: 0",axis = 1)

In [ ]:
# Example 2: Given Context

In [ ]:
temp.iloc[30,6]

"Throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with Destiny's Child, making her one of the best-selling music artists of all time. The Recording Industry Association of America recognized her as the Top Certified Artist in America during the 2000s decade. Time listed her among the 100 most influential people in the world in 2013 and 2014. In 2009, Billboard named her the Top Radio Songs Artist of the Decade, the Top Female Artist of the 2000s and their Artist of the Millennium in 2011"

In [ ]:
# Exmaple 2: Reference Context

In [ ]:
temp.iloc[30,7]

'On stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. A self-described "modern-day feminist", Beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. She has won 20 Grammy Awards and is the most nominated woman in the award\'s history. Forbes magazine also listed her as the most powerful female musician of 2015.'